In [1]:
import pandas as pd

In [2]:
llama = pd.read_csv("../results/llama_nb.csv")

In [3]:
openai = pd.read_csv("../results/scores_openai.csv")

In [35]:
llama["example_id"]

0       1a
1       1b
2       1c
3       1d
4       1a
      ... 
56    lbe2
57    lbe3
58    lbe4
59    lbe3
60    lbe4
Name: example_id, Length: 61, dtype: object

In [36]:
openai['id']

0       1a
1       1b
2       1c
3       1d
4       1a
      ... 
57    lbe2
58    lbe3
59    lbe4
60    lbe3
61    lbe4
Name: id, Length: 62, dtype: object

In [25]:
openai

,id,lang,sentence,condition,grammatical,source,gpt-4_score
0,1a,no,Hvilken artikkel kastet du uten å lese,both open,True,parasitic gaps,5
1,1b,no,Hvilken artikkel kastet du uten å lese den,parasitic plugged,True,parasitic gaps,5
2,1c,no,Hvilken artikkel kastet du den uten å lese,licensing plugged,False,parasitic gaps,2
3,1d,no,Hvilken artikkel kastet du den uten å lese den,both plugged,True,parasitic gaps,1
4,1a,en,Which article did you throw away without reading,both open,True,parasitic gaps,5
...,...,...,...,...,...,...,...
57,lbe2,no,Hvem sa hun at de malte et portrett av,complement extraction,True,lbe_test_set,5
58,lbe3,en,Who do you believe pictures of will be on sale,left branch extraction,False,lbe_test_set,2
59,lbe4,en,Who did she say portraits of hung in the gallery,left branch extraction,False,lbe_test_set,2
60,lbe3,no,Hvem tror du at bilder av vil bli lagt ut for ...,left branch extraction,True,lbe_test_set,1


In [40]:
llama['language'] = llama.lang.apply(lambda x: x[:2].lower())

In [61]:
res = llama.merge(openai, left_on=["example_id","language"], right_on=["id","lang"], how="outer")

In [62]:
res.score = res.fillna(1).score.astype(int)

In [63]:
res[res.score != res["gpt-4_score"]]

,example_id,sentence_x,lang_x,score,raw_output,language,id,lang_y,sentence_y,condition,grammatical,source,gpt-4_score
4,1c,Which article did you throw it away without re...,English,2,The sentence is mostly ungrammatical because '...,en,1c,en,Which article did you throw it away without re...,licensing plugged,False,parasitic gaps,1
5,1c,Hvilken artikkel kastet du den uten å lese,Norwegian,4,"The sentence is mostly grammatical, but it sou...",no,1c,no,Hvilken artikkel kastet du den uten å lese,licensing plugged,False,parasitic gaps,2
6,1d,Which article did you throw it away without re...,English,2,The sentence is mostly ungrammatical due to th...,en,1d,en,Which article did you throw it away without re...,both plugged,True,parasitic gaps,1
7,1d,Hvilken artikkel kastet du den uten å lese den,Norwegian,5,"The sentence is fully grammatical, with correc...",no,1d,no,Hvilken artikkel kastet du den uten å lese den,both plugged,True,parasitic gaps,1
8,2a,Which room did he clean before painting,English,4,The sentence is mostly grammatical but lacks a...,en,2a,en,Which room did he clean before painting,both open,True,parasitic gaps,5
12,2c,Which room did he clean it before painting,English,2,The sentence is mostly ungrammatical due to th...,en,2c,en,Which room did he clean it before painting,licensing plugged,False,parasitic gaps,1
13,2c,Hvilket rom ryddet han det før han malte,Norwegian,5,"The sentence is fully grammatical, with correc...",no,2c,no,Hvilket rom ryddet han det før han malte,licensing plugged,False,parasitic gaps,1
14,2d,Which room did he clean it before painting it,English,2,The sentence is mostly ungrammatical due to th...,en,2d,en,Which room did he clean it before painting it,both plugged,True,parasitic gaps,1
15,2d,Hvilket rom ryddet han det før han malte det,Norwegian,5,"The sentence is fully grammatical, with correc...",no,2d,no,Hvilket rom ryddet han det før han malte det,both plugged,True,parasitic gaps,1
16,3a,Ola is a man who everyone who plays chess with...,English,4,"The sentence is mostly grammatical, but the em...",en,3a,en,Ola is a man who everyone who plays chess with...,both open,True,parasitic gaps,5


In [64]:
res['diff'] = res.score - res["gpt-4_score"]

In [68]:
res["gram"] = res.grammatical.apply(lambda x: 5 if x else 1)

In [70]:
res[["id", "sentence_x", "lang_y", "score", "gpt-4_score","gram", "diff"]].to_csv('comparisons.csv')

In [69]:
res

,example_id,sentence_x,lang_x,score,raw_output,language,id,lang_y,sentence_y,condition,grammatical,source,gpt-4_score,diff,gram
0,1a,Which article did you throw away without reading,English,5,"The sentence is fully grammatical, with proper...",en,1a,en,Which article did you throw away without reading,both open,True,parasitic gaps,5,0,5
1,1a,Hvilken artikkel kastet du uten å lese,Norwegian,5,"The sentence is fully grammatical, with correc...",no,1a,no,Hvilken artikkel kastet du uten å lese,both open,True,parasitic gaps,5,0,5
2,1b,Which article did you throw away without readi...,English,4,"The sentence is mostly grammatical, but the pr...",en,1b,en,Which article did you throw away without readi...,parasitic plugged,True,parasitic gaps,4,0,5
3,1b,Hvilken artikkel kastet du uten å lese den,Norwegian,5,"The sentence is fully grammatical, with correc...",no,1b,no,Hvilken artikkel kastet du uten å lese den,parasitic plugged,True,parasitic gaps,5,0,5
4,1c,Which article did you throw it away without re...,English,2,The sentence is mostly ungrammatical because '...,en,1c,en,Which article did you throw it away without re...,licensing plugged,False,parasitic gaps,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,rel7,Gutten som så filmen gikk hjem,Norwegian,5,"The sentence is fully grammatical, following s...",no,rel7,no,Gutten som så filmen gikk hjem,declarative,True,relclause_wordorder_test_set,5,0,5
58,NaN,NaN,NaN,1,NaN,NaN,rel8,en,Has the boy who seen the movie will go home,Aux1-fronted (ungrammatical),False,relclause_aux_test_set_fixed,1,0,1
59,rel8,Så gutten som filmen gikk hjem,Norwegian,1,The sentence lacks a verb and has an unclear w...,no,rel8,no,Så gutten som filmen gikk hjem,V1-fronted (ungrammatical),False,relclause_wordorder_test_set,1,0,1
60,rel9,Will the boy who has seen the movie go home,English,5,"The sentence is fully grammatical, with proper...",en,rel9,en,Will the boy who has seen the movie go home,Aux2-fronted (grammatical),True,relclause_aux_test_set_fixed,5,0,5
